In [42]:
import os
import json
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import gensim

In [43]:
normalized_dir = "../polarities/normalized"

In [44]:
len(os.listdir(normalized_dir))

20

In [4]:
subreddits = list()

In [5]:
subreddit_polarities = dict()
for subreddit in os.listdir(normalized_dir):
    with open(os.path.join(normalized_dir,subreddit)) as f:
        subreddit_polarities[subreddit.split('_standardized')[0]] = json.load(f)
        subreddits.append(subreddit.split('_standardized')[0])

In [6]:
subreddit_polarities.keys()

dict_keys(['BingeEatingRecovery', 'AnorexiaRecovery', 'bingeeating', 'eating_disorders', 'EdAnonymousAdults', 'EatingDisorderHope', 'selftext', 'bulimia', 'BulimiaRecovery', 'EDRecovery_public', 'BingeEatingDisorder', 'EatingDisorders', 'anorexiaflareuphelp', 'edsupport', 'PurgingDisorder', 'AnorexiaNervosa', 'BulimiaAndAnaSupport', 'EDAnonymous', 'NotOtherwiseSpecified'])

In [7]:
subreddits

['BingeEatingRecovery',
 'AnorexiaRecovery',
 'bingeeating',
 'eating_disorders',
 'EdAnonymousAdults',
 'EatingDisorderHope',
 'selftext',
 'bulimia',
 'BulimiaRecovery',
 'EDRecovery_public',
 'BingeEatingDisorder',
 'EatingDisorders',
 'anorexiaflareuphelp',
 'edsupport',
 'PurgingDisorder',
 'AnorexiaNervosa',
 'BulimiaAndAnaSupport',
 'EDAnonymous',
 'NotOtherwiseSpecified']

In [8]:
sqlite_path = "../reddit_ed_0.6.0.sqlite3"
engine = create_engine(
    f"sqlite:///{sqlite_path}",
    connect_args={"check_same_thread":False},
)

In [9]:
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        
        if token not in stopwords.words('english'):
            
            result.append(token)
    return result

# Regular Summing

### Don't use

In [ ]:
polarity_dfs = dict()
for subreddit in subreddits:
    if subreddit != 'BingeEatingRecovery':
        polarities = list()
        print(subreddit)
        query = f"select id, selftext from subreddit_submission_metadata where subreddit='{subreddit}';"
        df = pd.read_sql_query(text(query),con=engine.connect())
        for sentence in df.values:
            preprocessed_sentence = preprocess(sentence[1])
        #     print(preprocessed_sentence)
            sentence_polarities = [subreddit_polarities[subreddit][word] for word in preprocessed_sentence if word in subreddit_polarities[subreddit].keys()]
        #     print(sentence_polarities)
            sum_polarities = np.sum(sentence_polarities)
        #     print(f"Regular sum of sentence non-stopwords polarities: {sum_polarities}")  
        #     print(sentence.append(sum_polarities))
            polarities.append({'id':sentence[0], 'selftext':sentence[1],'sentence_polarity':sum_polarities})


        polarity_dfs[subreddit] = pd.DataFrame.from_dict(polarities)


# Averaging Polarity Scores

better than regular summing

In [10]:
polarity_dfs = dict()
for subreddit in subreddits:
    if subreddit != 'BingeEatingRecovery':
        polarities = list()
        print(subreddit)
        query = f"select id, selftext from subreddit_submission_metadata where subreddit='{subreddit}';"
        df = pd.read_sql_query(text(query),con=engine.connect())
        for sentence in df.values:
            preprocessed_sentence = preprocess(sentence[1])
            sentence_polarities = [subreddit_polarities[subreddit][word] for word in preprocessed_sentence if word in subreddit_polarities[subreddit].keys()]
        #     print(sentence_polarities)
            avg_polarities = np.average(sentence_polarities)
        #     print(f"Regular sum of sentence non-stopwords polarities: {sum_polarities}")  
        #     print(sentence.append(sum_polarities))
            polarities.append({'id':sentence[0], 'selftext':sentence[1],'sentence_polarity':avg_polarities})


        polarity_dfs[subreddit] = pd.DataFrame.from_dict(polarities)

AnorexiaRecovery


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


bingeeating


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


eating_disorders


...

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


EDAnonymous


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


NotOtherwiseSpecified


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [11]:
polarity_dfs.keys()

dict_keys(['AnorexiaRecovery', 'bingeeating', 'eating_disorders', 'EdAnonymousAdults', 'EatingDisorderHope', 'selftext', 'bulimia', 'BulimiaRecovery', 'EDRecovery_public', 'BingeEatingDisorder', 'EatingDisorders', 'anorexiaflareuphelp', 'edsupport', 'PurgingDisorder', 'AnorexiaNervosa', 'BulimiaAndAnaSupport', 'EDAnonymous', 'NotOtherwiseSpecified'])

In [12]:
polarity_dfs["AnorexiaRecovery"]

,id,selftext,sentence_polarity
0,10pljws,Is it possible to gain 5 pounds in 10 days? I’...,0.355774
1,10pg79q,"Hello all, not sure if this is the best place ...",0.230702
2,10p9kga,I’m soo proud of myself 🥲,5.592233
3,10p8r2g,do i need to be concerned about chest pain whe...,0.508727
4,10p6vc9,"2 weeks ago is was 58 lbs, I got on the scale ...",0.453443
...,...,...,...
2681,g70olb,"I hope no one minds my updates, I promise they...",0.597833
2682,g6sef1,,NaN
2683,g6se8c,[deleted],-0.067754
2684,g6bt08,"Today I took my supplements, stayed hydrated a...",-0.619578


In [18]:
for subreddit in polarity_dfs.keys():
#     print(subreddit)
    sub = [subreddit for i in range(len(polarity_dfs[subreddit]))]
    polarity_dfs[subreddit]['subreddit'] = sub

In [21]:
all_subreddits_polarity_df = pd.concat([polarity_dfs[subreddit] for subreddit in polarity_dfs.keys()])

## Combined Sentence Polarity Scores for All Individual Subreddits Excluding BingeEatingRecovery

In [25]:
all_subreddits_polarity_df.groupby(by='subreddit')['id'].count()

subreddit
AnorexiaNervosa           977
AnorexiaRecovery         2686
BingeEatingDisorder       393
BulimiaAndAnaSupport      117
BulimiaRecovery            23
EDAnonymous              3910
EDRecovery_public         529
EatingDisorderHope         99
EatingDisorders           904
EdAnonymousAdults        4954
NotOtherwiseSpecified      18
PurgingDisorder            15
anorexiaflareuphelp        38
bingeeating               421
bulimia                  2326
eating_disorders         3663
edsupport                  83
Name: id, dtype: int64

In [29]:
all_subreddits_polarity_df.to_csv('submission_polarity.csv')